In [4]:

from collections import defaultdict


def processar_info_produtos_df(df):
    if df is None or df.empty:

        print("DataFrame de produtos está vazio ou não foi carregado. Não é possível processar.")
        return None

    info_produtos_processada = defaultdict(lambda: {
        "organico": False,  
        "sazonalidade": [], # 
        "avaliacoes_por_associacao": [], 
        "associacoes_ofertantes": set(), 
        "avaliacao_media": 3.0
    })


    colunas_esperadas = [
        "nome_associacao", "localidade_principal", "nome_produto", 
        "produto_organico", "avaliacao_produto", 
        "disponivel_primavera", "disponivel_verao", "disponivel_outono", "disponivel_inverno"
    ]

    for _, row in df.iterrows():

        if not all(col in row for col in colunas_esperadas):
            print(f"Atenção: Linha com dados ausentes ou colunas faltando (verifique se o CSV tem todas as colunas esperadas). Pulando processamento parcial: {row.to_dict()}")
            continue
            
        nome_produto = str(row["nome_produto"]).strip()
        if not nome_produto: 
            continue
        
        nome_assoc = str(row["nome_associacao"])

        if int(row["produto_organico"]) == 1:
            info_produtos_processada[nome_produto]["organico"] = True 
        
        sazonalidade_atual_produto = info_produtos_processada[nome_produto]["sazonalidade"]
        if int(row["disponivel_primavera"]) == 1 and "primavera" not in sazonalidade_atual_produto:
            sazonalidade_atual_produto.append("primavera")
        if int(row["disponivel_verao"]) == 1 and "verao" not in sazonalidade_atual_produto:
            sazonalidade_atual_produto.append("verao")
        if int(row["disponivel_outono"]) == 1 and "outono" not in sazonalidade_atual_produto:
            sazonalidade_atual_produto.append("outono")
        if int(row["disponivel_inverno"]) == 1 and "inverno" not in sazonalidade_atual_produto:
            sazonalidade_atual_produto.append("inverno")
        
        if pd.notna(row["avaliacao_produto"]):
            try:
                info_produtos_processada[nome_produto]["avaliacoes_por_associacao"].append(
                    (float(row["avaliacao_produto"]), nome_assoc) 
                )
            except ValueError:
                print(f"Aviso: Valor de avaliação inválido '{row['avaliacao_produto']}' para o produto '{nome_produto}' da associação '{nome_assoc}'. Ignorando esta avaliação.")

        info_produtos_processada[nome_produto]["associacoes_ofertantes"].add(nome_assoc)
            
    for produto, dados in info_produtos_processada.items():
        if dados["avaliacoes_por_associacao"]:
            soma_avaliacoes = sum(aval for aval, assoc in dados["avaliacoes_por_associacao"])
            dados["avaliacao_media"] = soma_avaliacoes / len(dados["avaliacoes_por_associacao"])
            
    return info_produtos_processada